In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os

In [2]:
def read_vocab(filename):
    
    dict_ = {}
    with open(filename, 'r', encoding='utf-8-sig') as datafile:
        for line in datafile:
            occurence = line.strip().split()[0]
            word = line.strip().split()[1]
            dict_[word] = int(occurence)
    return dict_

In [3]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/twitter_datasets_epfl/full/')

In [ ]:
#  vocab pos
os.system('build_vocab_pos_full.sh')
os.system('cut_vocab_pos_full.sh')

#  vocab neg
os.system('build_vocab_neg_full.sh')
os.system('cut_vocab_neg_full.sh')

In [4]:
# read the vocabs
vocab_neg = read_vocab('vocab_pos_full.txt')
vocab_pos = read_vocab('vocab_neg_full.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'cut_vocab_pos_full.txt'

In [ ]:
import operator

def extract_most_freq(vocab_dict, max_words=10):
    
    most_freq_words = []
    vocab_list_sorted = sorted(vocab_dict.copy().items(), key=operator.itemgetter(1), reverse = True)

    for i in range(max_words):
        most_freq_words.append((vocab_list_sorted[i][0], vocab_list_sorted[i][1]))
    
    return most_freq_words

In [ ]:
most_neg = extract_most_freq(vocab_neg, max_words=10)
print("\n".join("{}\t{}".format(x[0], x[1]) for x in most_neg))

In [ ]:
most_pos = extract_most_freq(vocab_pos, max_words=10)
print("\n".join("{}\t{}".format(x[0], x[1]) for x in most_pos))

In [ ]:
def word_ratio_neg_pos(vocab_dict_neg, vocab_dict_pos):
    
    word_ratio = {}

    # scan negative tweets
    for word_neg, occu_neg in vocab_dict_neg.items():  
        
        # if word in both neg and pos vocab
        if word_neg in vocab_dict_pos:
            
            # get occurence in positive tweets
            occu_pos = vocab_dict_pos.get(word_neg)
            
            # compute the ratio
            word_ratio[word_neg] = [occu_neg/occu_pos, occu_neg, occu_pos]
            
    return word_ratio

In [ ]:
freq_min = 1
vocab_neg_thr = {k: v for k, v in vocab_neg.copy().items() if v > freq_min}
vocab_pos_thr = {k: v for k, v in vocab_pos.copy().items() if v > freq_min}

In [ ]:
word_ratio = word_ratio_neg_pos(vocab_neg_thr, vocab_pos_thr)

In [ ]:
print(word_ratio.get('happy'))
print(word_ratio.get('not'))

In [ ]:
# build exclusion vocab with stop words
ratio_margin = 0.1
freq_min = 100
stop_vocab = {}

for word, values in word_ratio.items():
    
    ratio = values[0]
    occu_neg = values[1]
    occu_pos = values[2]
    
    # remove non discriminant word
    if ratio > 1 - ratio_margin and ratio < 1 + ratio_margin:
        stop_vocab[word] = values
        
    # remove unfrequent word
    if occu_neg < freq_min and occu_pos < freq_min:
        stop_vocab[word] = values
    
print("Number of stop words: ", len(stop_vocab))

In [ ]:
print("\n".join("{}\t{}".format(k,v) for k,v in stop_vocab.items()))

In [ ]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/stop_words/')

In [ ]:
# write stop word in a text
outputfilename = "stop_word_" + "freq_min_" + str(freq_min) + "_ratio_marg_" + str(ratio_margin) + ".txt"
with open(outputfilename, 'w', encoding='utf-8-sig') as outfile:
    for word, value in stop_vocab.items():
        outfile.write(word)   
        outfile.write("\n")      
outfile.close()